This is my Peer-graded Assignment for Segmenting and Clustering Neighborhoods in Toronto

This is to uninstall/install libraries to eliminate some errors.

In [1]:
!pip uninstall -y pandas
!pip install pandas
import pandas as pd
!pip uninstall -y numpy
!pip install numpy
import numpy as np
!pip uninstall -y scipy
!pip install scipy
print("Done.")

Found existing installation: pandas 1.2.3
Uninstalling pandas-1.2.3:
  Successfully uninstalled pandas-1.2.3
  Using cached pandas-1.2.3-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibm-watson-machine-learning 1.0.53 requires pandas<=1.0.5, but you have pandas 1.2.3 which is incompatible.
Found existing installation: numpy 1.20.1
Uninstalling numpy-1.20.1:
  Successfully uninstalled numpy-1.20.1
  Using cached numpy-1.20.1-cp37-cp37m-manylinux2010_x86_64.whl (15.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibm-watson-machine-learning 1.0.53 requires pandas<=1.0.5, but you have pandas 1.2.3 which is incompatible.
Found existing installation: scipy 1.6.1
Uninstalling scipy-1.6.1:
  Suc

In [2]:
!pip install bs4
from bs4 import BeautifulSoup # this module helps in web scrapping.

#!pip install requests
import requests  # this module helps us to download a web page

#import numpy as np # library to handle data in a vectorized manner
#import pandas as pd

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# KMeans from sklearn
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1902
  - defaults/noarch::ibm-wsrt-py37main-main==custom=1902
done

# All requested packages already installed.

Libraries imported.


The following cell is for webscraping.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

#find a html table in the web page
tables = soup.find_all('table') # in html table is represented by the tag <table>
for index,table in enumerate(tables):
    if ("Neighbourhood" in str(table)):
        table_index = index
#print(tables[table_index].prettify())


The following cell is to convert the data into a pandas dataframe.

In [4]:
dfpostal = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        PostalCode = col[0].text.strip()
        Borough= col[1].text.strip()
        if col[2].text.strip() == "Not assigned":
            Neighborhood = Borough
        else:
            Neighborhood = col[2].text.strip()
        if Borough != "Not assigned":
            dfpostal = dfpostal.append({"PostalCode":PostalCode, "Borough":Borough, "Neighborhood":Neighborhood}, ignore_index=True)

dfpostal.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
dfpostal.shape

(103, 3)

This is to get the geographical coordinates of each postal code.

In [6]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [7]:
dflocation = pd.read_csv('Geospatial_data.csv')
dflocation.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
dflocation.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


This is to merge the dataframes.

In [8]:
dftoronto = pd.merge(dfpostal, dflocation, 
                     left_on = 'PostalCode', 
                     right_on = 'PostalCode', 
                     how='left')
dftoronto.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [9]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of', address, 'are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postalcode, borough, neighborhood in zip(dftoronto['Latitude'], dftoronto['Longitude'], dftoronto['PostalCode'], dftoronto['Borough'], dftoronto['Neighborhood']):
    label = '{}, {}'.format(postalcode, borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

This is to define my Foursquare credentials

In [12]:
{
    "tags": [
        "remove-cell",
    ]
}

CLIENT_ID = 'RYEYR1MIOM31BKZPPGHTY50N21G3OGEICV0HRT2MO5TMOJO4' # your Foursquare ID
CLIENT_SECRET = 'MCFTLL2ZC4U5KIJAYHJC33S4PI33O5CJH2HVIKR21IVR4CPA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RYEYR1MIOM31BKZPPGHTY50N21G3OGEICV0HRT2MO5TMOJO4
CLIENT_SECRET:MCFTLL2ZC4U5KIJAYHJC33S4PI33O5CJH2HVIKR21IVR4CPA


In [190]:
def getNearbyVenues(names, latitudes, longitudes, radius=1200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
print("Function defined")

Function defined


In [191]:
{
    "tags": [
        "hide-output",
    ]
}
toronto_venues = getNearbyVenues(names=dftoronto['PostalCode'],
                                   latitudes=dftoronto['Latitude'],
                                   longitudes=dftoronto['Longitude'])


M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [192]:
print(toronto_venues.shape)
toronto_venues.head()
#toronto_venues['Venue Category'].unique()
toronto_venues.loc[toronto_venues['PostalCode'] == 'M1X']

(5807, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5299,M1X,43.836125,-79.205636,Beige-ville,43.8451,-79.208755,Donut Shop


In [193]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode']

# move PostalCode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#print(fixed_columns)
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,PostalCode,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [194]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.041667
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.022222,0.0,0.022222,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M9N,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
99,M9P,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
100,M9R,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
101,M9V,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [195]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [196]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Coffee Shop,Trail,Chinese Restaurant,Bank,Big Box Store,Supermarket,Spa,Bus Station,Caribbean Restaurant
1,M1C,Breakfast Spot,Italian Restaurant,Gym / Fitness Center,Burger Joint,Playground,Park,Opera House,Pakistani Restaurant,Paintball Field,Outdoors & Recreation
2,M1E,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Liquor Store,Park,Gym / Fitness Center,Greek Restaurant,Shopping Mall,Beer Store
3,M1G,Park,Coffee Shop,Indian Restaurant,Pizza Place,Supermarket,Thrift / Vintage Store,Supplement Shop,Fast Food Restaurant,Juice Bar,Sandwich Place
4,M1H,Coffee Shop,Indian Restaurant,Gas Station,Bakery,Bank,Restaurant,Sandwich Place,Grocery Store,Pizza Place,Bar


This is to apply the KMeans algorithm for clustering.

In [197]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#print(kmeans.labels_[0:100])

# add clustering labels
if 'Cluster Labels' in neighborhoods_venues_sorted.columns:
    neighborhoods_venues_sorted.drop('Cluster Labels', 1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.sort_values('PostalCode')

#neighborhoods_venues_sorted[neighborhoods_venues_sorted.isna().any(axis=1)]

#neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['PostalCode'] == 'M1X']

#print(neighborhoods_venues_sorted['Cluster Labels'].to_string(index=False))

,Cluster Labels,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,M1B,Fast Food Restaurant,Coffee Shop,Trail,Chinese Restaurant,Bank,Big Box Store,Supermarket,Spa,Bus Station,Caribbean Restaurant
1,0,M1C,Breakfast Spot,Italian Restaurant,Gym / Fitness Center,Burger Joint,Playground,Park,Opera House,Pakistani Restaurant,Paintball Field,Outdoors & Recreation
2,1,M1E,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Liquor Store,Park,Gym / Fitness Center,Greek Restaurant,Shopping Mall,Beer Store
3,1,M1G,Park,Coffee Shop,Indian Restaurant,Pizza Place,Supermarket,Thrift / Vintage Store,Supplement Shop,Fast Food Restaurant,Juice Bar,Sandwich Place
4,1,M1H,Coffee Shop,Indian Restaurant,Gas Station,Bakery,Bank,Restaurant,Sandwich Place,Grocery Store,Pizza Place,Bar
...,...,...,...,...,...,...,...,...,...,...,...,...
98,1,M9N,Furniture / Home Store,Train Station,Park,Skating Rink,Grocery Store,Food & Drink Shop,Café,Gas Station,Sandwich Place,Laundromat
99,1,M9P,Pizza Place,Gas Station,Coffee Shop,Intersection,Skating Rink,Golf Driving Range,Golf Course,Middle Eastern Restaurant,Supermarket,Fried Chicken Joint
100,1,M9R,Coffee Shop,Pizza Place,Pharmacy,Sandwich Place,Bank,Chinese Restaurant,Shopping Mall,Baseball Field,Beer Store,Grocery Store
101,1,M9V,Pizza Place,Grocery Store,Coffee Shop,Park,Fried Chicken Joint,Caribbean Restaurant,Bus Line,Fast Food Restaurant,Café,Auto Garage


In [200]:
toronto_merged = dftoronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.sort_values('PostalCode')
#toronto_merged[toronto_merged.isna().any(axis=1)]
#print(toronto_merged['Cluster Labels'].to_string(index=False))

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Coffee Shop,Trail,Chinese Restaurant,Bank,Big Box Store,Supermarket,Spa,Bus Station,Caribbean Restaurant
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Breakfast Spot,Italian Restaurant,Gym / Fitness Center,Burger Joint,Playground,Park,Opera House,Pakistani Restaurant,Paintball Field,Outdoors & Recreation
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Liquor Store,Park,Gym / Fitness Center,Greek Restaurant,Shopping Mall,Beer Store
22,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Park,Coffee Shop,Indian Restaurant,Pizza Place,Supermarket,Thrift / Vintage Store,Supplement Shop,Fast Food Restaurant,Juice Bar,Sandwich Place
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Coffee Shop,Indian Restaurant,Gas Station,Bakery,Bank,Restaurant,Sandwich Place,Grocery Store,Pizza Place,Bar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,M9N,York,Weston,43.706876,-79.518188,1,Furniture / Home Store,Train Station,Park,Skating Rink,Grocery Store,Food & Drink Shop,Café,Gas Station,Sandwich Place,Laundromat
70,M9P,Etobicoke,Westmount,43.696319,-79.532242,1,Pizza Place,Gas Station,Coffee Shop,Intersection,Skating Rink,Golf Driving Range,Golf Course,Middle Eastern Restaurant,Supermarket,Fried Chicken Joint
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,1,Coffee Shop,Pizza Place,Pharmacy,Sandwich Place,Bank,Chinese Restaurant,Shopping Mall,Baseball Field,Beer Store,Grocery Store
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,1,Pizza Place,Grocery Store,Coffee Shop,Park,Fried Chicken Joint,Caribbean Restaurant,Bus Line,Fast Food Restaurant,Café,Auto Garage


In [201]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [202]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,0,Breakfast Spot,Italian Restaurant,Gym / Fitness Center,Burger Joint,Playground,Park,Opera House,Pakistani Restaurant,Paintball Field,Outdoors & Recreation
45,North York,0,Park,Coffee Shop,Pub,Intersection,Restaurant,Japanese Restaurant,Gym,Gym / Fitness Center,Sushi Restaurant,Noodle House
46,North York,0,Park,Grocery Store,Vietnamese Restaurant,Bank,Shopping Mall,Coffee Shop,Pizza Place,Moving Target,Poke Place,Pakistani Restaurant
50,North York,0,Park,Sports Bar,Bakery,Shopping Mall,Bank,Skating Rink,Pharmacy,Electronics Store,Italian Restaurant,Neighborhood
58,Scarborough,0,Park,Gym,Café,Convenience Store,Ice Cream Shop,Restaurant,Gym Pool,General Entertainment,College Stadium,Skating Rink
88,Etobicoke,0,Park,Mexican Restaurant,Indian Restaurant,Pizza Place,Supermarket,Café,Fast Food Restaurant,Skating Rink,Grocery Store,Bakery
101,Etobicoke,0,Park,Coffee Shop,Italian Restaurant,Pizza Place,Thai Restaurant,Eastern European Restaurant,Shopping Mall,Harbor / Marina,Fast Food Restaurant,Bank


### Cluster 2

In [204]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Park,Pharmacy,Convenience Store,Discount Store,Bus Stop,Café,Skating Rink,Caribbean Restaurant,Shop & Service,Chinese Restaurant
5,Etobicoke,1,Pharmacy,Grocery Store,Bus Stop,Bakery,Shopping Mall,Golf Course,Park,Skating Rink,Café,Playground
6,Scarborough,1,Fast Food Restaurant,Coffee Shop,Trail,Chinese Restaurant,Bank,Big Box Store,Supermarket,Spa,Bus Station,Caribbean Restaurant
8,East York,1,Pharmacy,Intersection,Athletics & Sports,Bakery,Skating Rink,Park,Pizza Place,Burger Joint,Café,Soccer Stadium
10,North York,1,Fast Food Restaurant,Grocery Store,Coffee Shop,Pizza Place,Bakery,Sandwich Place,Park,Bank,Discount Store,Gas Station
17,Etobicoke,1,Coffee Shop,Pizza Place,Baseball Field,Park,Pet Store,Convenience Store,Flower Shop,Garden,Gas Station,Café
18,Scarborough,1,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Liquor Store,Park,Gym / Fitness Center,Greek Restaurant,Shopping Mall,Beer Store
21,York,1,Discount Store,Japanese Restaurant,Grocery Store,Mexican Restaurant,Beer Store,Bus Stop,Coffee Shop,Park,Pharmacy,ATM
22,Scarborough,1,Park,Coffee Shop,Indian Restaurant,Pizza Place,Supermarket,Thrift / Vintage Store,Supplement Shop,Fast Food Restaurant,Juice Bar,Sandwich Place
26,Scarborough,1,Coffee Shop,Indian Restaurant,Gas Station,Bakery,Bank,Restaurant,Sandwich Place,Grocery Store,Pizza Place,Bar


### Cluster 3

In [205]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Scarborough,2,Donut Shop,Noodle House,Pakistani Restaurant,Paintball Field,Outdoors & Recreation,Other Great Outdoors,Organic Grocery,Optical Shop,Opera House,Office


### Cluster 4

In [206]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,3,Hotel,Pizza Place,Park,Grocery Store,Pharmacy,Mexican Restaurant,Fish & Chips Shop,Gym,Restaurant,Gift Shop
60,North York,3,Hotel,Discount Store,Pizza Place,Coffee Shop,Gas Station,Pharmacy,Grocery Store,Theater,Vietnamese Restaurant,Fast Food Restaurant
94,Etobicoke,3,Hotel,Moving Target,Dog Run,Mediterranean Restaurant,Coffee Shop,Music Store,Music Venue,Paintball Field,Museum,Outdoors & Recreation


### Cluster 5

In [207]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,4,Coffee Shop,Pizza Place,Indian Restaurant,Thrift / Vintage Store,Portuguese Restaurant,Playground,Sandwich Place,Optical Shop,French Restaurant,Middle Eastern Restaurant
2,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Bakery,Theater,Gastropub,Park,Farmers Market,Hotel,Seafood Restaurant
3,North York,4,Clothing Store,Restaurant,Coffee Shop,Fast Food Restaurant,Dessert Shop,Furniture / Home Store,Men's Store,Chocolate Shop,Fried Chicken Joint,Sushi Restaurant
4,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Park,Ramen Restaurant,Clothing Store,Thai Restaurant,Café,Japanese Restaurant,Italian Restaurant,Dance Studio
7,North York,4,Coffee Shop,Japanese Restaurant,Italian Restaurant,Bank,Pharmacy,Sandwich Place,Fried Chicken Joint,Burger Joint,Liquor Store,Pizza Place
9,Downtown Toronto,4,Coffee Shop,Gastropub,Japanese Restaurant,Café,Theater,Seafood Restaurant,Italian Restaurant,Creperie,Diner,Cosmetics Shop
13,North York,4,Restaurant,Coffee Shop,Gym,Beer Store,Movie Theater,Pizza Place,Sushi Restaurant,Grocery Store,Park,Supermarket
14,East York,4,Coffee Shop,Café,Park,Pizza Place,Arts & Crafts Store,Bar,Sandwich Place,Ice Cream Shop,Thai Restaurant,Intersection
15,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Seafood Restaurant,Theater,Gastropub,Italian Restaurant,Hotel,Farmers Market,American Restaurant
16,York,4,Coffee Shop,Caribbean Restaurant,Bank,Convenience Store,Pizza Place,Deli / Bodega,Dance Studio,Beer Store,Mexican Restaurant,Grocery Store
